In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import pandas as pd
import re
import datetime
import numpy as np
import pickle
import requests
from time import sleep

In [3]:
version = "1.0.0"

# Get The Article list

In [1]:


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import ui
from selenium.webdriver.support.ui import WebDriverWait, Select

from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
driver.set_window_size(1920, 1080)



In [40]:
def gedjournalarticles(journal, filename ):
    start_url = "https://academic.oup.com/journals/advanced-search"
    driver.get(start_url)
    print(driver.current_url)
    sleep(4)
    
    content = driver.page_source
    
    # open old file
    olddf = pd.DataFrame({'url': [], 'journal_title': [],
                                'title': [], 'writer': [], 
                                'doi': [], 'abstract': [],
                                'keyword': [], 'publishdate': [],
                                'language': [],
                                'category': [],
                                'journal_eissn': [], 'journal_pissn': []})
    try:
        olddf = pd.read_csv(filename, sep="|")
        print(olddf.tail())
    except:
        pass

    # set journal
    # publisher = driver.find_element_by_id("docId")
    # publisher.send_keys(journal['eissn'])
    selectpublisher = Select(driver.find_element_by_id("SelectedJournal"))
    print(journal['title'].split(":")[0])
    selectpublisher.select_by_visible_text(journal['title'].split(":")[0])
    driver.find_element_by_id("btnAdvancedCitationFilter").click()
    sleep(4)

    # just for Research articles
    try:
        driver.find_element_by_id("article-type-label0").click()
    except Exception as e:
        print(e)        
    
    sleep(4)
    print(driver.current_url)
    
    # BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, features="lxml")
    
    # get the articles 
    articles = soup.find_all('div', {'class': ['al-article-box']})
    
    # open all of the abstract
    for selabstract in driver.find_elements_by_class_name('js-show-abstract'):
        selabstract.click()
    
    
    for article in articles:
        title = article.find("h4").get_text()
        url = article.find("h4").find("a")['href']
        date = article.find('div', {'class': ['al-pub-date']}).get_text().replace("Published: ", '')
        date = datetime.datetime.strptime(date, "%d %B %Y").strftime('%Y-%m-%d')        
        abstract = article.find('section', {'class': ['abstract']}).get_text()      
        doi = ""
        doidiv = article.find('div', {'class': ['al-citation-list']}).find_all('a')
        for a in doidiv:
            if re.search("doi.org", a["href"] ):
                doi = a["href"].replace("https://doi.org/", '')
        writers = []
        writersdeiv =  article.find('div', {'class': ['al-authors-list']}).find_all( 'span', {'class': ['wi-fullname']} )
        for w in writersdeiv:
            writers.append(w.get_text())
        writers = "#".join(writers)
        
        
        print(title, url, date, abstract, doi, writers)
      
    
    """
    return df, olddf
    """

# Main

In [32]:
journals = pd.read_csv("doaj_journals.csv", sep="|")
journals = journals[journals['publisher'] == "Oxford University Press"]
journals = journals.to_dict('records')

In [42]:
jidx = 0
filename = filename = 'data/journal_'+ journals[jidx]['title'].replace(" ", "_").replace(":", "").replace("&", "and") +'_'+version+'.csv'
gedjournalarticles( journals[jidx], filename )

https://academic.oup.com/crawlprevention/governor?content=%2fjournals%2fadvanced-search


NoSuchElementException: Message: Unable to locate element: [id="SelectedJournal"]


In [4]:


for jidx in range(len(journals)):
    filename = filename = 'data/journal_'+ journals[jidx]['title'].replace(" ", "_").replace(":", "").replace("&", "and") +'_'+version+'.csv'
    
    # search for articles
    df, olddf = gedjournalarticles( journals[jidx], filename )
    
    # get the articles details
    getarticledetails(df, olddf, filename)



NameError: name 'journals' is not defined